# Optimalizace kódu

**Spoiler warning:** Tento notebook obsahuje téměř komplentní řešení. Nechcete-li se připravit o potěšení z tutoriálu, použijte jen bezpečnou verzi [optimalizace-student.ipynb](optimalizace-student.ipynb).

Cílem je vytvořit agenta, kterému dáte funkci v C/C++, návod jak vygenerovat vstup, a agent vám tuto funkci skompiluje, změří její rychlost, a navrhne optimalizované řešení. S agentem lze pracovat iterativně, kdy mu radíte, co ještě vyzkoušet, a agent bude dále kompilovat a spouštět alternativní řešení.

Kód kompilujte jednoduše, ať nemusíte instalovat fancy věci -- s gcc lze použít např.
```
g++ -O3 -o bench bench.cpp -fopenmp
```
Jazykovému modelu je pak vhodné prozradit, co smí používat, či přímo jak kód kompilujete. Pokročilejší verze pak může nechat LLM měnit způsob kompilace.

Agenta testujte na něčem jednoduchém. Aby se jeho doladěnější verze trochu "zabavily", použijte už složitější kód Coulombovské sumace:
```
struct sAtom{
    float x, y, z;
    float w;
};
void coulombNaive(const sAtom* __restrict__ atoms, const int numberOfAtoms, const float gridSpacing, const int gridPoints, float* __restrict__ grid) {
    for (int a = 0; a < numberOfAtoms; a++) {
        sAtom myAtom = atoms[a];
        for (int x = 0; x < gridPoints; x++) {
            float dx2 = powf((float)x * gridSpacing - myAtom.x, 2.0f);
            for (int y = 0; y < gridPoints; y++) {
                float dy2 = powf((float)y * gridSpacing - myAtom.y, 2.0f);
                for (int z = 0; z < gridPoints; z++) {
                    float dz = (float)z * gridSpacing - myAtom.z;
                    float e = myAtom.w / sqrtf(dx2 + dy2 + dz*dz);
                    grid[z*gridPoints*gridPoints + y*gridPoints + x] += e;
                }
            }
        }
    }
```
Do pole atoms můžete pro testování nasypat náhodná čísla, ideálně v intervalu (0, 1), gridSpacing nastavte na 0.01. Dostatečně dlouhý výpočet chce pole grid alokované na velikost 256x256x256, tím pádem gridPoints nastavte na 256, počet atomů numberOfAtoms nastavte okolo 1000 a adekvátně tomu alokujte pole atoms. Pole grid alokujte vynulované.

**Upozornění** v doporučené velikosti vstupu trvá naivní implementace několik minut, nicméně optimalizované verze lze zrychlit na běh v řádu nízkých jednotek sekund na běžném notebooku, proto je kvůli přesnosti měření doporučen relativně velký vstup.

In [ ]:
from openai import OpenAI
import json
import subprocess
import os
from openai import ChatCompletion

In [ ]:
#API key
OPENAI_API_KEY = "TODO"

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def compiler(source):
    """
    Write source code into a file and compile it into a binary.
    Args:
        source (str): the string that is written and compiled.
    Returns:
        output (str): Compilation output from g++ compiler.    
    """
    with open("bench.cpp", "w") as text_file:
        text_file.write(source)
    result = subprocess.run(["/usr/bin/g++", "-O3", "-o", "bench", "bench.cpp", "-fopenmp"], stdout=subprocess.PIPE)
    return result.stdout

In [ ]:
def executor():
    """
    Execute the source file (binary bench) and return its benchmarked runtime

    Returns:
        output (str): Output of the benchmarked code.
    """

    result = subprocess.run(['./bench',], stdout=subprocess.PIPE)
    return result.stdout

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": compiler.__name__,
        "description": compiler.__doc__,
        "parameters": {
            "type": "object",
            "properties": {
                "source": {"type": "string"},
            },
            "required": ["source"],
            "additionalProperties": False
        },
        "strict": True
    }
},
{
    "type": "function",
    "function": {
        "name": executor.__name__,
        "description": executor.__doc__,
        "parameters": {
            "type": "object",
            "properties": {},
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [ ]:
SYSTEM_PROMPT = "You are a tool for optimization of the code. The user gives you a C or C++ function that is to be optimized. The user specifies also how to generate input for the function. Your role is to create an optimized code of the function. You have to create a code that generates input for the function, as well as the code that measures its runtime in microseconds and dumps it to the standard output. Do not use any fancy features like TBB, only stuff that goes with GCC by default, such as OpenMP. If you generate multiple versions of the code, use the same method of measuring time to have comparable results."
ASSISTANT_PROMPT = "First, prepare the source code benchmarking the unmodified function, benchmark it, and let the user know the result. Then, optimize the function, and do the same for the optimized one."

In [ ]:
class Agent():
    def __init__(
        self,
        system_prompt: str,
        assistant_prompt: str,
        model: str,
        tools: list
    ):

        self._system_prompt = system_prompt
        self._assistant_prompt = assistant_prompt
        self._messages = []
        if system_prompt is not None:
            self._messages.append({
                "role": "system",
                "content": system_prompt
            })
        if assistant_prompt is not None:
            self._messages.append({
                "role": "assistant",
                "content": assistant_prompt
            })
        self._model = model
        self._tools = tools

    def _parse_response(self, response: ChatCompletion) -> dict:
        # check if we have a tool call
        if response.choices[0].finish_reason == 'tool_calls':
            tool_calls =  response.choices[0].message.tool_calls
            next_step = {
                'target': 'llm',
                'messages': [],
            }

            for tool_call in tool_calls:
                args = json.loads(tool_call.function.arguments)
                # check what function to call
                if tool_call.function.name == 'compiler':
                    result = compiler(**args)
                    next_step["messages"].append({
                        'id': tool_call.id,
                        'name': tool_call.function.name,
                        'args': args,
                        'content': str(result)
                    })
                elif tool_call.function.name == 'executor':
                    result = executor(**args)
                    next_step["messages"].append({
                        'id': tool_call.id,
                        'name': tool_call.function.name,
                        'args': args,
                        'content': str(result)
                    })
                
                else:
                    next_step["messages"].append({
                        'id': tool_call.id,
                        'name': tool_call.function.name,
                        'args': args,
                        'content': "Unsupported function"
                    })

        else:
            next_step = {
                "target": "user",
                "messages": [response.choices[0].message.content]
            }

        return next_step

    def flush(self):
        self._messages = []
        if self._system_prompt is not None:
            self._messages.append({
                "role": "system",
                "content": self._system_prompt
            })
        if self._assistant_prompt is not None:
            self._messages.append({
                "role": "system",
                "content": self._assistant_prompt
            })

    def run(self, query: str) -> str:
        # add the new user query to our messages
        self._messages.append({
            "role": "user",
            "content": query
        })
        for step in range(10):
            print(f"Agent step: {step}")
            try:
                response = client.chat.completions.create(
                    model=self._model,
                    messages=self._messages,
                    tools=self._tools,
                    timeout=100,
                    max_completion_tokens=1000
                )
            except Exception as e:
                print(f"Request failed: {e}")
                print(f"Trying again")
                continue

            print(f"Response: ", response)

            next_step = self._parse_response(response)

            if next_step["target"] == "user":
                return next_step['messages'][0]
            
            self._messages.append(response.choices[0].message)
            for tool_call in next_step['messages']:
                print(f"Called tool: {tool_call['name']}, with arguments: {tool_call['args']}")

                print(f"Tool output: {tool_call['content']}\n")

                self._messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call["id"],
                    "content": tool_call["content"]
                })
        
        # The agent kept looping
        return "The agent could not come up with an answer to the query"

In [ ]:
agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    assistant_prompt=ASSISTANT_PROMPT,
    model="gpt-4o-mini",
    tools=tools
)

In [ ]:
agent.flush()
agent.run("My function is as follows: double sum(const double* x, int n) {double tmp = 0.0; for (int i = 0; i < n; i++) tmp += x[i]; return tmp;}. Test the function on an input vector of size 100000000 elements, filled by random numbers in the interval from 0.0 to 1.0.")

In [ ]:
agent.run("Try another optimization trick, for example, loop unrolling, while disabling OpenMP.")

In [ ]:
agent.run("Perfect, this helped. Maybe try to combine it with OpenMP.")

In [ ]:
agent.flush()
agent.run("My function is as follows: "
"struct sAtom{"
"    float x, y, z;"
"    float w;"
"};"
"void coulombNaive(const sAtom* __restrict__ atoms, const int numberOfAtoms, const float gridSpacing, const int gridPoints, float* __restrict__ grid) {"
"    for (int a = 0; a < numberOfAtoms; a++) {"
"        sAtom myAtom = atoms[a];"
"        for (int x = 0; x < gridPoints; x++) {"
"            float dx2 = powf((float)x * gridSpacing - myAtom.x, 2.0f);"
"            for (int y = 0; y < gridPoints; y++) {"
"                float dy2 = powf((float)y * gridSpacing - myAtom.y, 2.0f);"
"                for (int z = 0; z < gridPoints; z++) {"
"                    float dz = (float)z * gridSpacing - myAtom.z;"
"                    float e = myAtom.w / sqrtf(dx2 + dy2 + dz*dz);"
"                    grid[z*gridPoints*gridPoints + y*gridPoints + x] += e;"
"                }"
"            }"
"        }"
"    }"
"} Test the function on an input vector atoms of size 1000 elements, filled by random numbers in the interval from 0.0 to 1.0 in all x,y,z,w. Set numberOfAtoms to 1000, gridSpacing to 1.0, gridPoints to 256, and allocate grid size to 16777216 and fill it by zeroes."
"Generate new lines instead of \n in the output code, so it can be compiled. When calling malloc for atoms, be careful to retype it to *sAtom.")